In [ ]:
!pip install transformers
import pandas as pd
import tensorflow as tf
import transformers
from transformers import DistilBertTokenizer
from transformers import TFDistilBertForSequenceClassification
pd.set_option('display.max_colwidth', None)
MODEL_NAME = 'distilbert-base-uncased-finetuned-sst-2-english'
BATCH_SIZE = 16
N_EPOCHS = 3

     |████████████████████████████████| 3.4 MB 4.2 MB/s 
     |████████████████████████████████| 596 kB 50.9 MB/s 
     |████████████████████████████████| 895 kB 51.6 MB/s 
     |████████████████████████████████| 3.3 MB 28.9 MB/s 
     |████████████████████████████████| 61 kB 458 kB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
import pandas as pd
df = pd.read_csv('AbstractDataset_trainSet_FULL.csv')
df.head()

,id,file_name,text,#label
0,1,2111.09381_originalAbstract.txt,"We present MEDCOD, a Medically- Accurate, Emotive, Diverse, and Controllable Dialog system with a unique approach to the natural language generator module. MEDCOD has been developed and evaluated specifically for the history taking task. It integrates the advantage of a traditional modular approach to incorporate (medical) domain knowledge with modern deep learning techniques to generate flexible, human-like natural language expres- sions. Two key aspects of MEDCOD’s natural language output are described in detail. First, the generated sentences are emotive and em- pathetic, similar to how a doctor would com- municate to the patient. Second, the gen- erated sentence structures and phrasings are varied and diverse while maintaining medical consistency with the desired medical concept (provided by the dialogue manager module of MEDCOD). Experimental results demonstrate the effectiveness of our approach in creating a human-like medical dialogue system. Rele- vant code is available at https://github.com/ curai/curai- research/tree/main/MEDCOD",1
1,2,2111.09381_generatedAbstract.txt,"We present MEDCOD, a Medically- Accurate, Emotive, Diverse, and Controllable Dialog system with a unique approach to the natural language generator module. MEDCOD has been developed and evaluated specifically for the history taking task. It integrates the advantage of a traditional modular approach to incorporate (medical) domain knowledge with modern deep learning techniques to generate flexible, human-like natural language expres- sions. Two key aspects of MEDCOD’s natural language output are described in detail. First, the generated sentences are emotive and em- pathetic, similar to how a doctor would com- municate to the patient. Second, the gen- erated sentence structures and phrasings are varied and diverse while maintaining medical consistency with the desired medical concept (provided by the dialogue manager module of MEDCOD). Experimental results demonstrate that using this system improves the natural language generation system by at least 5% on both training and testing. Rele- vant code is available at https://github.com/ curai/curai- research/tree/main/MEDCOD",0
2,3,2111.09388_originalAbstract.txt,"This work applies Minimum Bayes Risk (MBR) decoding to optimize diverse auto- mated metrics of translation quality. Au- tomatic metrics in machine translation have made tremendous progress recently. In par- ticular, neural metrics, fine-tuned on human ratings (e.g. BLEURT, or COMET) are out- performing surface metrics in terms of cor- relations to human judgements. Our experi- ments show that the combination of a neural translation model with a neural reference- based metric, BLEURT, results in significant improvement in automatic and human eval- uations. This improvement is obtained with translations different from classical beam- search output: these translations have much lower likelihood and are less favored by sur- face metrics like BLEU.",1
3,4,2111.09388_generatedAbstract.txt,"This work applies Minimum Bayes Risk (MBR) decoding to optimize diverse auto- mated metrics of translation quality. Au- tomatic metrics in machine translation have made tremendous progress recently. In par- ticular, neural metrics, fine-tuned on human ratings (e.g. BLEURT, or COMET) are out- performing surface metrics in terms of cor- relations to human judgements. Our experi- ments show that these models can learn to approximate to human-defined representations with low performance . This improvement is obtained with translations different from classical beam- search output: these translations have much lower likelihood and are less favored by sur- face metrics like BLEU.",0
4,5,2111.09412_originalAbstract.txt,"In this study, we present a meta-learning model to adapt the predictions of the network’s capacity between viewers who participate in a live video streaming event. We propose the MELANIE model, whe

In [ ]:
X = df['text']
y = df['#label']
df.shape

(200, 4)

In [ ]:
messages = df.copy()
messages.reset_index(inplace = True)

In [ ]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size = 0.2,random_state = 0)

In [ ]:
#define a tokenizer object
tokenizer = DistilBertTokenizer.from_pretrained(MODEL_NAME)
#tokenize the text
train_encodings = tokenizer(list(X_train.values),
                            truncation=True, 
                            padding=True)
test_encodings = tokenizer(list(X_test.values),
                           truncation=True, 
                           padding=True)

In [ ]:
train_dataset = tf.data.Dataset.from_tensor_slices((dict(train_encodings),
                                    list(y_train.values)))
test_dataset = tf.data.Dataset.from_tensor_slices((dict(test_encodings),
                                    list(y_test.values)))

In [ ]:
model = TFDistilBertForSequenceClassification.from_pretrained(MODEL_NAME)
#chose the optimizer
optimizerr = tf.keras.optimizers.Adam(learning_rate=5e-5)
#define the loss function 
losss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
#build the model
model.compile(optimizer=optimizerr,
              loss=losss,
              metrics=['accuracy'])
# train the model 
model.fit(train_dataset.shuffle(len(X_train)).batch(BATCH_SIZE),
          epochs=N_EPOCHS,
          batch_size=BATCH_SIZE)

Downloading:   0%|          | 0.00/256M [00:00<?, ?B/s]

All model checkpoint layers were used when initializing TFDistilBertForSequenceClassification.

All the layers of TFDistilBertForSequenceClassification were initialized from the model checkpoint at distilbert-base-uncased-finetuned-sst-2-english.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFDistilBertForSequenceClassification for predictions without further training.


Epoch 1/3
10/10 [==============================] - 358s 34s/step - loss: 1.5845 - accuracy: 0.4187
Epoch 2/3
10/10 [==============================] - 337s 34s/step - loss: 0.7227 - accuracy: 0.5250
Epoch 3/3
10/10 [==============================] - 341s 34s/step - loss: 0.7015 - accuracy: 0.5000


In [ ]:
# model evaluation on the test set
model.evaluate(test_dataset.shuffle(len(X_test)).batch(BATCH_SIZE), 
               return_dict=True, 
               batch_size=BATCH_SIZE)

3/3 [==============================] - 50s 16s/step - loss: 0.6843 - accuracy: 0.6250


{'accuracy': 0.625, 'loss': 0.6842538714408875}